In [34]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')

df.head()

question      answer
0                   What is the capital of France?       Paris
1                  What is the capital of Germany?      Berlin
2               Who wrote 'To Kill a Mockingbird'?  Harper-Lee
3  What is the largest planet in our solar system?     Jupiter
4   What is the boiling point of water in Celsius?         100

In [35]:
# tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [36]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [37]:
# vocab
vocab = {'<UNK>':0}

In [38]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer

  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [39]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [40]:
len(vocab)

324

In [41]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [42]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [43]:
import torch
from torch.utils.data import Dataset, DataLoader

In [44]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):

    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [45]:
dataset = QADataset(df, vocab)

In [46]:
dataset[10]               #question numberr and answer

(tensor([ 1,  2,  3,  4,  5, 53]), tensor([54]))

In [47]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [48]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([233])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 10,  75, 208]]) tensor([209])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([16])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[ 42, 101,   2,   3,  17]]) tensor([102])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([36])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[ 42, 299, 300, 118,  14, 301, 302, 158, 303, 304, 305, 306]]) tensor([307])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([295])
tensor([[ 10,  11, 189, 158, 190]]) tensor([191])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([49])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([52

In [49]:
import torch.nn as nn

In [50]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [51]:
dataset[0][0]

tensor([1, 2, 3, 4, 5, 6])

In [52]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [53]:
learning_rate = 0.001
epochs = 20

In [54]:
model = SimpleRNN(len(vocab))

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 10.061260
Epoch: 2, Loss: 8.834305
Epoch: 3, Loss: 7.797769
Epoch: 4, Loss: 6.945150
Epoch: 5, Loss: 6.208784
Epoch: 6, Loss: 5.593142
Epoch: 7, Loss: 5.059387
Epoch: 8, Loss: 4.584542
Epoch: 9, Loss: 4.177839
Epoch: 10, Loss: 3.818292
Epoch: 11, Loss: 3.518420
Epoch: 12, Loss: 3.236193
Epoch: 13, Loss: 2.989289
Epoch: 14, Loss: 2.758131
Epoch: 15, Loss: 2.554419
Epoch: 16, Loss: 2.377810
Epoch: 17, Loss: 2.210574
Epoch: 18, Loss: 2.054097
Epoch: 19, Loss: 1.921748
Epoch: 20, Loss: 1.796461


In [57]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [67]:
predict(model, "What is the largest planet in our  system?")

I don't know
jupiter


In [59]:
list(vocab.keys())[7]

'paris'